In [175]:
import numpy as np

In [176]:
class TicTacToeSmall():
    
    def __init__(self):
        self.board = np.zeros((3, 3))
        
    def _check_align(self, array):
        return 0 not in array and np.unique(array).size == 1
        
    def _eval_board(self):
        
        # 3 circles / crosses align in row
        for row in self.board: 
            if self._check_align(row): return 'win'
        
        # 3 circles / crosses align in col
        for col in self.board.T: 
            if self._check_align(col): return 'win'
        
        # 3 circles / crosses align in diagonal
        diag = np.diagonal(self.board)
        if self._check_align(diag): return 'win'
        
        # 3 circles / crosses align in opposite diagonal
        oppo_diag = np.diagonal(np.fliplr(self.board))
        if self._check_align(oppo_diag): return 'win'
        
        # game board filled
        if 0 not in np.unique(self.board): return 'draw'
        
        return 'continue'
    
    def step(self, action, side):
        
        assert self.board[action] == 0, 'the position is filled'
        
        self.side = side
        
        if side == 'circle':
            self.board[action] = -1
        elif side == 'cross' :
            self.board[action] = 1
        
        self.progress = self._eval_board()
        if self.progress == 'continue':
            reward = 0
            done = False
        elif self.progress == 'win':
            reward = 1
            done = True
        elif self.progress == 'draw':
            reward = 0
            done = True
        
        return self.board, reward, done
    
    @property
    def action_space(self):
        advanced_indices = np.where(self.board == 0)
        actions = []
        for x, y in zip(advanced_indices[0], advanced_indices[1]):
            actions.append((x, y))
        return actions

In [177]:
def switch_player(player):
    if player == 'circle': return 'cross'
    elif player == 'cross': return 'circle'

In [182]:
def board_to_str(board):
        board_str = ''
        for i, row in enumerate(board):
            for item in row:
                if item == 0:
                    board_str += ' '
                elif item == -1:
                    board_str += 'o'
                elif item == 1:
                    board_str += 'x'
            if i < 2:
                board_str += '\n'
        return board_str

## Random round

In [191]:
game = TicTacToeSmall()
done = False
player = np.random.choice(['circle', 'cross'])

while done != True:
    action_index = np.random.randint(len(game.action_space))
    action = game.action_space[action_index]
    board, reward, done = game.step(action, player)
    player = switch_player(player)

print(board_to_str(game.board))
print(f'Last player: {game.side}')
if game.progress == 'draw':
    print(f'Game progress: {game.progress}')
elif game.progress == 'win':
    print(f'Game progress: {game.side} {game.progress}s')

x  
x  
xoo
Last player: cross
Game progress: cross wins


## Tabular value lookup + MCTS ()
- How MCTS works, what is the algorithm?
- How these two work together? Make tabular value and neural network modular

## Neural network as value approximator + MCTS